In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
stud=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w26/Part 1 Output.csv')
info=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w26/Additional Info Lookup.csv')

In [3]:
stud['Initials']=stud['Full Name'].apply(lambda x:"".join([word[0] for word in x.split()]) ).apply(lambda x:x[0:2])
stud['Date of Birth']=pd.to_datetime(stud['Date of Birth'])
info['Date of Birth']=pd.to_datetime(info['Date of Birth'])

<ipython-input-3-df907d2991f7>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  stud['Date of Birth']=pd.to_datetime(stud['Date of Birth'])


In [4]:
x=stud.merge(info,how='inner',on=['Initials','School Name','Maths','English','Science'])
x=x.drop_duplicates(subset=['Student ID']).reset_index().drop('index',axis=1)
x=x[['Student ID', 'Full Name', 'Date of Birth_x', 'Region', 'School Name',
       'School ID', 'English', 'Science', 'Maths', 'Grade Score', 'Initials','Home Address', 'Distance From School (Miles)',
       'Subject Selection']]

In [5]:
x['rank']=x.groupby(['Region','Subject Selection'])['Grade Score'].rank('first',ascending=False)

In [6]:
g=pd.DataFrame([],columns=x.columns)
h=pd.DataFrame([],columns=x.columns)
for a in (x['Subject Selection'].unique()) :
    for c in x.Region.unique():
        if c=="EAST":
            #print(a,c)
            f=(x[(x['Region']=='EAST')&(x['Subject Selection']==a)&(x['rank']<=15)])
            g=pd.concat([f,g])
        else:
            t=(x[(x['Region']=='WEST')&(x['Subject Selection']==a)&(x['rank']<=5)])
            h=pd.concat([t,h])

In [7]:
l=['Student ID', 'Full Name', 'Date of Birth_x', 'Region', 'School Name', 'English', 'Science', 'Maths', 'Grade Score', 'Subject Selection']
select=pd.concat([g,h])
select=select[l].reset_index().drop('index',axis=1)

In [8]:
select

,Student ID,Full Name,Date of Birth_x,Region,School Name,English,Science,Maths,Grade Score,Subject Selection
0,17893,REMINGTON PEARMAN,2007-01-23,EAST,Southbridge Secondary,A,A,A,150,Engineering
1,18822,WAVERLY TUTILL,2007-07-07,EAST,Waterview Academy,A,B,B,130,Engineering
2,18884,ISA MACDOWAL,2007-04-19,EAST,Waterview Academy,A,B,A,140,Engineering
3,17150,NEILLA FIDO,2007-06-23,EAST,Vista Academy,C,A,A,130,Engineering
4,18634,DORALYN CHAWKLEY,2007-05-17,EAST,Waterview Academy,A,B,B,130,Engineering
...,...,...,...,...,...,...,...,...,...,...
95,15681,CYMBRE NAIRY,2007-04-13,WEST,St. Mary's,B,A,A,140,Business Management
96,15804,KLARA MALY,2007-06-26,WEST,Eight Kings,A,A,A,150,Business Management
97,16894,GLEN LIBREY,2007-12-03,WEST,Riverbank School,A,A,A,150,Business Management
98,16340,FRANCISCA PLAISTOWE,2007-04-23,WEST,Long Road School,A,A,A,150,Business Management


In [9]:
p=((select.groupby(['Region','School Name'])['Full Name'].count()/select.groupby(['Region'])['Full Name'].count())*100).round(1).reset_index(name='Percentage')
y=select.groupby(['Region','School Name'])['Full Name'].count().reset_index(name='Count of Seats')

In [10]:
c=p.merge(y,how='inner',on=['Region','School Name'])

In [11]:
c.groupby('Region')['Percentage'].agg([min,max]).reset_index()

,Region,min,max
0,EAST,6.7,26.7
1,WEST,8.0,28.0


#### End Of Code